In [8]:
from __future__ import print_function 
import os, sys, h5py
import pandas as pd
import numpy as np
from six.moves import cPickle
from sklearn.metrics import roc_curve, auc, precision_recall_curve, accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline
data_path = '/home/peter/Code/tensorflow/data'
results_path = os.path.join(data_path, 'results', 'tfomics')

In [9]:

filename = 'valideval_dataset.hdf5'
dataset = h5py.File(os.path.join(data_path,filename),'r')
group_name = ['valid_data']
val_dat = np.array(dataset['/'+group_name[0]+'/vs_valid'])
val_lbl = np.array(dataset['/'+group_name[0]+'/label_valid'])

fragLen = 330
N = 14
startgap = np.ceil(float(val_dat.shape[1] - fragLen)/N).astype('int')
true_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')

cnt_ = 0
for a in range(val_dat.shape[0]):
    for b in range(val_dat.shape[0]):

        # Keep track of the true labels
        if val_lbl[a,b] == 1:
            true_lbl[cnt_] = 1
        else:
            true_lbl[cnt_] = 0

        cnt_ += 1        

In [11]:
files = ['resid_model_old_data_1.pickle', 
         'resid_model_old_data_2.pickle', 
         'resid_model_deepomics_old_data_1.pickle',
         #'resid_model_new_data_2.pickle',
         #'resid_model_new_data_3.pickle',
         #'resid_model_new_data_4.pickle'
        ]

pred_lbl = []
for filename in files:
    savefile = os.path.join(results_path, filename)
    f = open(savefile, 'rb')
    pred_lbl.append(cPickle.load(f))
    f.close()

# Tim's model
filename = 'predictions_dataset2_unscattered_1d_version_residual2_100_1000.txt'
savefile = os.path.join(results_path, filename)
df = pd.read_table(savefile, header=None)
pred_lbl.append(df[0].as_matrix())
pred_lbl = np.array(pred_lbl).T    

num_models = pred_lbl.shape[1]
    
for i in range(num_models):
    print('model ' + str(i))
    fpr, tpr, thresholds = roc_curve(true_lbl, pred_lbl[:,i])
    print('auc: ' + str(auc(fpr, tpr)))
    
fpr, tpr, thresholds = roc_curve(true_lbl, np.mean(pred_lbl, axis=1))
print('equally-weighted ensemble')
print('auc: ' + str(auc(fpr, tpr)))

model 0
auc: 0.935207313228
model 1
auc: 0.934452035981
model 2
auc: 0.937158958118
model 3
auc: 0.941078560941
equally-weighted ensemble
auc: 0.939903254381


# random weight sampling

In [12]:
patience = 500
best_results = 0
best_weights = 0

global_counter = 0
counter = 0
status = True
while status:    
    if np.mod(global_counter, 100) == 0:
        print('iterations: ' + str(global_counter))
    w = np.random.rand(num_models)
    w /= np.sum(w)
    
    prediction = 0
    for i in range(pred_lbl.shape[1]):
        prediction += pred_lbl[:,i]*w[i]

    fpr, tpr, thresholds = roc_curve(true_lbl, prediction)
    score = auc(fpr, tpr)
    
    if score > best_results:
        print('higher auc found: ' + str(score))
        best_weights = w
        best_results = score
        counter = 0
    counter += 1
    global_counter += 1
    if counter >= patience:
        status = False
        print('Finished searching')
        
print('AUC: ' + str(best_results))
print(best_weights)

iterations: 0
higher auc found: 0.9408367994
higher auc found: 0.941433504281
higher auc found: 0.941533728628


KeyboardInterrupt: 

# Gaussian process weight sampling

In [5]:
patience = 500
best_results = 0

w = np.random.rand(num_models)
w /= np.sum(w)
best_weights = w
sigma = .1

global_counter = 0
counter = 0
status = True
while status:    
    if np.mod(global_counter, 100) == 0:
        print('iterations: ' + str(global_counter))
    w = np.zeros(num_models)
    for i in range(num_models):
        w[i] = np.maximum(best_weights[i] + sigma*np.random.randn(), 0.0001)
    w /= np.sum(w)
    
    prediction = 0
    for i in range(pred_lbl.shape[1]):
        prediction += pred_lbl[:,i]*w[i]

    fpr, tpr, thresholds = roc_curve(true_lbl, prediction)
    score = auc(fpr, tpr)
    
    if score > best_results:
        print('higher auc found: ' + str(score))
        best_weights = w
        best_results = score
        counter = 0
    counter += 1
    global_counter += 1
    if counter >= patience:
        status = False
        print('Finished searching')
        
    if np.mod(counter, 100) == 0:
        sigma /= 2
        
print('AUC: ' + str(best_results))
print(best_weights)

iterations: 0
higher auc found: 0.937717655856
higher auc found: 0.938368220881
higher auc found: 0.939245758602
higher auc found: 0.939981228139
higher auc found: 0.940512686993
higher auc found: 0.941124040019
higher auc found: 0.941142254845
higher auc found: 0.941259979691
higher auc found: 0.941429054778
higher auc found: 0.941530011442
higher auc found: 0.941539268592
higher auc found: 0.941591231684
higher auc found: 0.941597865265
higher auc found: 0.941686756594
higher auc found: 0.941705926212
higher auc found: 0.941707799983
higher auc found: 0.941713470074
higher auc found: 0.941713885776
iterations: 100
higher auc found: 0.941716732247
iterations: 200
higher auc found: 0.941720486441
higher auc found: 0.941720668838
iterations: 300
higher auc found: 0.941721119587
higher auc found: 0.941721280752
iterations: 400
higher auc found: 0.941721395699
iterations: 500


KeyboardInterrupt: 

In [ ]:

def test_prediction(nntrainer, data_path):

    filename = 'competition_dataset_downsampled.hdf5'
    dataset = h5py.File(os.path.join(data_path,filename),'r')
    group_name = ['competition_data']
    val_dat = np.array(dataset['/'+group_name[0]+'/realval'])
    val_lbl = np.array(dataset['/'+group_name[0]+'/realtest'])

    fragLen = 330
    N = 14
    avg_F = np.mean(val_dat,axis=0)

    startgap = np.ceil(float(val_dat.shape[1] - fragLen)/N).astype('int')
    true_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')
    pred_lbl = np.zeros((val_dat.shape[0]*val_dat.shape[0],), dtype='float32')

    # Counter for the "true_lbl" array
    cnt_ = 0
    # Counter for the "pred_lbl" array
    cnt_u = 0
    for a in range(val_dat.shape[0]):
        if a%100 == 0:
            print('\r' + 'X'*(a//100))

        # Create batch array to send thru network
        im_eval = np.empty((N*val_dat.shape[0],3,fragLen,1), dtype='float32')

        # Count the number of traces in each batch
        cnt = 0

        for b in range(val_dat.shape[0]):

            for n in range(0, val_dat.shape[1] - fragLen, startgap):
                try:
                    im_eval[cnt,:,:,0] = np.vstack((val_dat[a,n:n+fragLen],
                                         val_dat[b,n:n+fragLen],
                                         avg_F[n:n+fragLen]))
                except:
                    from IPython.core.debugger import Tracer
                    Tracer()()

                cnt += 1

            # Keep track of the true labels
            if val_lbl[a,b] == 1:
                true_lbl[cnt_] = 1
            else:
                true_lbl[cnt_] = 0

            cnt_ += 1

        # Run batch through network
        test = {'inputs': im_eval, 'keep_prob_conv': 1, 'keep_prob_dense': 1, 'is_training': False}
        pred_stop = nntrainer.get_activations(test, layer='output')[:,0]
        # Average output over each group of N traces
        for u in range(0, len(pred_stop), N):
            pred_lbl[cnt_u] = np.mean(pred_stop[u:u+N])
            cnt_u += 1        
    return pred_lbl


nntrainer.set_best_parameters()
pred_lbl = test_prediction(nntrainer, data_path)